In [15]:
%reload_ext autoreload
%autoreload 2
## sys package
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"  # specify which GPU(s) to be used
sys.path.append("./prediction_models/input/prostate-cancer-grade-assessment/")
## warning off
import warnings
warnings.filterwarnings("ignore")

## general package
import random
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import torch
from torch.utils.data import *
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from scipy.special import expit
# device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu") 
# print(device)

In [16]:
csv_file = "./csv_pkl_files/4_fold_whole_train_wo_sus.csv"
df = pd.read_csv(csv_file, index_col = False)
df = df.set_index("image_id")
df.head()

,Unnamed: 0,Unnamed: 0.1,data_provider,isup_grade,gleason_score,tile_number,split,tile_pixel,tile_blueratio
image_id,,,,,,,,,
9716b5d2e13a28be3dc836134153720c,0,0,radboud,4,4+4,7,1.0,"121,109,111,86,81,79,77,","48,40,46,33,32,33,28,"
0954f0bb123cf3be8b00c9c989dcf3d6,1,1,karolinska,0,0+0,43,1.0,"152,141,135,134,127,128,128,123,126,122,117,11...","47,41,43,40,45,44,39,43,45,43,48,42,40,33,41,3..."
d861180850ca54282b38c782734a7dc7,2,2,karolinska,0,0+0,39,0.0,"89,82,72,67,70,69,69,60,61,69,68,69,58,57,60,6...","32,30,29,29,25,27,26,26,26,25,25,25,26,25,26,2..."
459154b1334d073cfef8af95362a7051,3,3,karolinska,0,0+0,34,1.0,"112,103,102,99,92,89,97,90,94,88,83,83,82,79,7...","42,34,40,33,37,31,38,31,33,33,29,30,33,31,30,2..."
27489a7f1972687cccb1258116ff7f10,4,4,karolinska,0,0+0,24,0.0,"105,101,101,92,87,85,84,89,86,83,81,77,73,80,7...","34,35,37,35,32,31,32,29,31,29,31,28,28,29,27,2..."


In [17]:
df['split'] = np.random.permutation(df['split'])

In [18]:
with open('./csv_pkl_files/extracted_image_features.pkl', 'rb') as handle:
    image_features = pickle.load(handle)

In [19]:
fold = 3
df_train = df.loc[df["split"] != 0]
df_test = df.loc[df["split"] == 0]
train_features, train_label = [],[]
valid_features, valid_label = [],[]
for i in df_train.index:
    train_features.append(image_features[i])
    train_label.append(df.loc[i, "isup_grade"])
for i in df_test.index:
    valid_features.append(image_features[i])
    valid_label.append(df.loc[i, "isup_grade"])
    
train_features, valid_features = np.concatenate(train_features,0), np.concatenate(valid_features,0)
# train_features, valid_features = np.asarray(train_features), np.asarray(valid_features)
train_label, valid_label = np.asarray(train_label), np.asarray(valid_label)

In [20]:
params = {
    'application': 'regression',
    'boosting': 'gbdt',
    'metric': 'rmse',
    'num_leaves': 102,
    'learning_rate': 0.02,
    'bagging_fraction': 0.9951448659512921,
    'bagging_freq': 3,
    'feature_fraction': 0.6867901263802068,
    'verbosity': -1,
    'early_stop': 100,
    'verbose_eval': 1000,
    'num_rounds': 10000,
    'raw_seed': 1234,
    'max_bin': 127,
    'min_child_samples': 38,
    'lambda_l1': 0.42651295024341174,
    'lambda_l2': 0.15395842517107572,
    'max_depth': 14,
    'min_split_gain': 0.023658591149106636
}

In [21]:
def QWK(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.rint(preds)
    score = cohen_kappa_score(preds, labels, weights='quadratic')
    return ("QWK", score, True)

In [24]:
dtrain = lgb.Dataset(train_features, train_label)
dvalid = lgb.Dataset(valid_features, valid_label)

num_rounds = params['num_rounds']
verbose_eval = params['verbose_eval']
early_stop = params['early_stop']

model = lgb.train(params,
                  train_set=dtrain,
#                   valid_sets=(dtrain, dvalid),
                  valid_sets = dtrain,
                  num_boost_round=num_rounds,
                  verbose_eval=verbose_eval,
                  early_stopping_rounds=early_stop,
                  feval = QWK)

Training until validation scores don't improve for 100 rounds
[1000]	training's rmse: 0.113858	training's QWK: 0.998624
[2000]	training's rmse: 0.113858	training's QWK: 0.998624
[3000]	training's rmse: 0.113858	training's QWK: 0.998624
[4000]	training's rmse: 0.113858	training's QWK: 0.998624
[5000]	training's rmse: 0.113858	training's QWK: 0.998624
[6000]	training's rmse: 0.113858	training's QWK: 0.998624
[7000]	training's rmse: 0.113858	training's QWK: 0.998624
[8000]	training's rmse: 0.113858	training's QWK: 0.998624
[9000]	training's rmse: 0.113858	training's QWK: 0.998624
[10000]	training's rmse: 0.113858	training's QWK: 0.998624
Did not meet early stopping. Best iteration is:
[1024]	training's rmse: 0.113858	training's QWK: 0.998624


In [25]:
def fix_distribution(y_train, pred):
    """
    Convert predictions to labels such that the labels have 
    the same distribution as in the y_train array.
    """
    base = pd.Series([0, 0, 0, 0, 0, 0], index=np.arange(0, 6))
    thresholds = (base + pd.Series(y_train).value_counts()).fillna(0).cumsum()
    thresholds = thresholds / len(y_train) * len(pred)
    
    pred_ranks = pd.Series(pred).rank()
    ranked_scores = np.zeros(len(pred))

    for j, threshold in list(enumerate(thresholds))[::-1]:
        ranked_scores[pred_ranks <= threshold] = j
    return ranked_scores

In [28]:
preds = model.predict(valid_features)
preds_e = fix_distribution(train_label, preds)

In [29]:
kappa = cohen_kappa_score(valid_label, preds_e, weights='quadratic')
print(kappa)

0.9565674264214278


In [365]:
model.save_model(f"./csv_pkl_files/lgb_{fold}.txt")

In [7]:
model = lgb.Booster(model_file=f"./csv_pkl_files/lgb_{fold}.txt")

In [10]:
preds = model.predict(valid_features)
preds_e = fix_distribution(train_label, preds)

In [11]:
kappa = cohen_kappa_score(valid_label, preds_e, weights='quadratic')
print(kappa)

0.9831477942453927


In [13]:
valid_label, preds_e

(array([4, 0, 5, ..., 0, 5, 1]), array([4., 0., 5., ..., 0., 5., 1.]))